In [17]:
import numpy as np
import cv2 as cv
from skimage import feature
from sklearn.decomposition import PCA

In [3]:
# retrieve the resized images using numpy
flood_images_resized_blur = np.load('../dataset/dataset/flooded_resized_blur.npy')
non_flood_images_resized_blur = np.load('../dataset/dataset/non-flooded_resized_blur.npy')

In [4]:
# print the shape of the numpy images 
print(flood_images_resized_blur.shape)
print(non_flood_images_resized_blur.shape)

(461, 256, 256, 3)
(461, 256, 256, 3)


## Feauture Extraction

**Harris Corner Detection**

In [5]:
# # use Harris Corner Detection to extract features from the images
# def harris_corner_detection(img):
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     gray = np.float32(gray)
#     dst = cv.cornerHarris(gray, 2, 3, 0.04)
#     dst = cv.dilate(dst, None)
#     ret, dst = cv.threshold(dst, 0.01*dst.max(), 255, 0)
#     dst = np.uint8(dst)
#     ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)
#     criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
#     corners = cv.cornerSubPix(gray, np.float32(centroids), (5, 5), (-1, -1), criteria)
#     # flatten corners
#     corners = corners.flatten()
#     print(corners.shape)
#     return corners

# # extract the features from the images
# flood_features = []
# non_flood_features = []
# for i in range(len(flood_images_resized_blur)):
#     flood_features.append(harris_corner_detection(flood_images_resized_blur[i]))
#     break
# for i in range(len(non_flood_images_resized_blur)):
#     non_flood_features.append(harris_corner_detection(non_flood_images_resized_blur[i]))
#     break



**sift**

In [6]:
# # use sift to extract features from the images
# def sift(img):
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     sift = cv.SIFT_create()
#     kp, des = sift.detectAndCompute(gray, None)
#     # flatten des
#     #des = des.flatten()
#     print(des.shape)
#     return des

# # extract the features from the images
# flood_features = []
# non_flood_features = []
# for i in range(len(flood_images_resized_blur)):
#     flood_features.append(sift(flood_images_resized_blur[i]))
#     break
# for i in range(len(non_flood_images_resized_blur)):
#     non_flood_features.append(sift(non_flood_images_resized_blur[i]))
#     break



**orb**

In [7]:
# # use orb to extract features from the images  
# def surf(img):
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     orb = cv.ORB_create()
#     kp, des = orb.detectAndCompute(gray, None)
#     # flatten des
#     #des = des.flatten()
#     print(des.shape)
#     return des

# # extract the features from the images
# flood_features = []
# non_flood_features = []
# for i in range(len(flood_images_resized_blur)):
#     flood_features.append(surf(flood_images_resized_blur[i]))
#     break
# for i in range(len(non_flood_images_resized_blur)):
#     non_flood_features.append(surf(non_flood_images_resized_blur[i]))
#     break

**Histogram of Oriented Gradients (HOG)**

In [36]:
# use Histogram of Oriented Gradients (HOG) to extract features from the images
def hog(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    winSize = (128,128)
    blockSize = (32,32)
    blockStride = (16,16)
    cellSize = (16,16)
    nbins = 9
    hog = cv.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    h = hog.compute(gray)
    #print(h.shape)
    return h

# extract the features from the images
flood_features_hog = []
# flood_features_hog_2 = []
non_flood_features_hog = []
# non_flood_features_hog_2 = []
for i in range(len(non_flood_images_resized_blur)):
    flood_features_hog.append(hog(flood_images_resized_blur[i]))
    non_flood_features_hog.append(hog(non_flood_images_resized_blur[i]))
    #break

# for i in range(256, len(non_flood_images_resized_blur)):
#     flood_features_hog_2.append(hog(flood_images_resized_blur[i]))
#     non_flood_features_hog_2.append(hog(non_flood_images_resized_blur[i]))
#     break

# turn the features into numpy arrays
flood_features_hog = np.array(flood_features_hog)# + flood_features_hog_2)
non_flood_features_hog = np.array(non_flood_features_hog)# + non_flood_features_hog_2)

# print the shape of the numpy arrays
print(flood_features_hog.shape)
print(non_flood_features_hog.shape)



(461, 142884)
(461, 142884)


In [37]:
# save the features to numpy files
np.save('../dataset/dataset/flooded_features_hog.npy', flood_features_hog)
np.save('../dataset/dataset/non-flooded_features_hog.npy', non_flood_features_hog)

**Local Binary Patterns (LBP)**

In [13]:
# use Local Binary Patterns (LBP). to extract features from the images using skimage
def lbp_skimage(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, 24, 8, method="uniform")
    # flatten lbp
    lbp = lbp.flatten()
    #print(lbp.shape)
    return lbp

# extract the features from the images
flood_features_lbp = []
non_flood_features_lbp = []
for i in range(len(flood_images_resized_blur)):
    flood_features_lbp.append(lbp_skimage(flood_images_resized_blur[i]))

for i in range(len(non_flood_images_resized_blur)):
    non_flood_features_lbp.append(lbp_skimage(non_flood_images_resized_blur[i]))
    

print(flood_features_lbp[0].shape)
print(non_flood_features_lbp[0].shape)

(65536,)
(65536,)


In [14]:
# save the features to numpy files
np.save('../dataset/dataset/flooded_features_lbp.npy', flood_features_lbp)
np.save('../dataset/dataset/non-flooded_features_lbp.npy', non_flood_features_lbp)

## Feature Selection

**PCA**

In [27]:
# flood_features_lbp = np.array(flood_features_lbp)
# non_flood_features_lbp = np.array(non_flood_features_lbp)



In [30]:
# pca = PCA(n_components=461)
# # pca.fit(flood_features_lbp)
# # variance = 0
# # i = 0
# # while variance < 0.95:
# #     variance += pca.explained_variance_ratio_[i]
# #     i += 1
# # print(i)
# # pca = PCA(n_components=i)
# flood_features_lbp_ = pca.fit_transform(flood_features_lbp)
# print(flood_features_lbp_.shape)


(461, 461)


In [31]:
# pca = PCA(n_components=461  )
# # pca.fit(non_flood_features_lbp)
# # # loop until the variance is 75%
# # variance = 0
# # i = 0
# # while variance < 1:
# #     variance += pca.explained_variance_ratio_[i]
# #     i += 1
# # print(i)
# # pca = PCA(n_components=i)
# non_flood_features_lbp_ = pca.fit_transform(non_flood_features_lbp)
# print(non_flood_features_lbp_.shape)


(461, 461)


In [32]:
# # save the features to numpy files
# np.save('../dataset/dataset/flooded_features_lbp_pca.npy', flood_features_lbp_)
# np.save('../dataset/dataset/non-flooded_features_lbp_pca.npy', non_flood_features_lbp_)